In [1]:
from typing import List, Optional
from pathlib import Path
from collections import defaultdict

In [2]:
import pandas as pd
import numpy as np

In [3]:
dataframes = sorted(Path('../results').glob('*.csv'))

In [4]:
key_columns = ['제목', '방송 시작']
columns = ['제목', '방송 기간', '방송 횟수', '장르', '채널', '제작사', '극본', '출연자', '방송 시작']

In [5]:
frames = {
    data.stem: pd.read_csv(str(data))
    for data in dataframes
    if data.stem != 'results'
}

for frame in frames.values():
    assert (frame.columns == columns).all()

In [6]:
total = defaultdict(set)

In [7]:
for key, frame in frames.items():
    for _, args in frame[key_columns].iterrows():
        total[tuple(args.values)].add(key)

## Merge multiple domain dataframes

In [8]:
def merge(values: List[str]) -> str:
    """Simple merge return maximum length data
    """
    def valuable(value: np.ndarray) -> int:
        item = value.item()
        if isinstance(item, str):
            return len(item)
        return 0
    
    index = np.array(tuple(map(valuable, values))).argmax()
    
    return values[index].item()

In [9]:
table = pd.DataFrame([], columns=columns)

In [10]:
for key, domains in total.items():
    if len(domains) > 1:
        table = table.append(pd.Series([merge([frames[domain].loc[(frames[domain][key_columns] == key).all(axis=1)][column].values for domain in domains]) for column in columns], index=columns), ignore_index=True)
    else:
        domain, *_ = domains
        table = table.append(frames[domain].loc[(frames[domain][key_columns] == key).all(axis=1)])

In [11]:
print(table.shape)
table.head()

(1909, 9)


,제목,방송 기간,방송 횟수,장르,채널,제작사,극본,출연자,방송 시작
0,4월의키스,2004년 4월 21일 ~ 2004년 7월 8일,24부작,"멜로, 드라마",한국방송공사,CAA,박범수,"수애, 조한선, 이정진, 소이현 外",2004-04-21 00:00:00
1,으라차차와이키키2,2019년 3월 25일 ~ 2019년 5월 14일,16부작,드라마,JTBC,"씨제스엔터테인먼트, 드라마하우스앤드제이콘텐트허브 백창주, 박진형, 박준서","김기호, 송지은, 송미소, 서동범","김선호 (배우), 이이경, 신현수 (배우), 문가영, 안소희, 김예원 (1987년) 외",2019-03-25 00:00:00
2,타인은지옥이다,2019년 8월 31일 ~ 2019년 10월 6일,10부작,"공포, 스릴러",OCN (방송),"김우상, 권미경 영화사 우상, 스튜디오N",정이도,"임시완, 이동욱, 이정은(1970) 外",2019-08-31 00:00:00
3,마을-아치아라의비밀,2015년 10월 7일 ~ 2015년 12월 3일,16부작,"서스펜스 스릴러, 공포, 판타지",SBS (대한민국의 방송사),김동래 래몽래인,도현정,"문근영, 육성재, 온주완, 신은경 (배우), 장희진 (1983년) 외",2015-10-07 00:00:00
4,하이에나,ON AIR 2020년 2월 21일 ~,부작,NaN,SBS | 넷플릭스,SBS | 키이스트,김루리,"김혜수, 주지훈 外",unknown


In [12]:
table.to_csv('../results/results.csv', index=None)